# Emotion Chat Bot

In [42]:
from operator import itemgetter
from pprint import pprint

import torch
from libs import (
    EmotionModel,
    ResponseGeneratorPipeline,
    SimilarityAnalyser,
    generate_dummy_representation,
    get_sentiment_composition,
)
from sympy.core.random import randint
from torch import Tensor
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TextStreamer,
    pipeline,
)
from unsloth import FastLanguageModel

## Load Each Module

### Response Generator

In [43]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="hermeschen1116/response_generator_for_emotion_chat_bot",
    attn_implementation="flash_attention_2",
    pretraining_tp=1,
    load_in_4bit=True,
    device_map="auto",
    low_cpu_mem_usage=True,
    trust_remote_code=True,
)

Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.9: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.754 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '
<string>:209: SyntaxWarning: invalid escape sequence '\ '
<string>:210: SyntaxWarning: invalid escape sequence '\_'
<string>:211: SyntaxWarning: invalid escape sequence '\ '


In [44]:
response_generator = ResponseGeneratorPipeline(
    model,
    tokenizer,
    framework="pt",
    task="conversation-generation",
    num_workers=16,
    torch_dtype="auto",
    add_special_tokens=True,
    truncation=False,
    padding=True,
)

In [45]:
FastLanguageModel.for_inference(response_generator.model)
response_generator.model = torch.compile(
    response_generator.model, mode="reduce-overhead"
)

### Sentiment Analyzer

In [46]:
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [47]:
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/sentiment_analysis_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [48]:
sentiment_analyzer = pipeline(
    "sentiment-analysis",
    model=sentiment_analysis_model,
    tokenizer=sentiment_analysis_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [49]:
sentiment_analyzer.model = torch.compile(
    sentiment_analyzer.model, mode="reduce-overhead"
)

In [50]:
print(sentiment_analyzer.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Predictor

In [51]:
emotion_predictor_model = AutoModelForSequenceClassification.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16
    ),
    device_map="auto",
    low_cpu_mem_usage=True,
)

In [52]:
emotion_predictor_tokenizer = AutoTokenizer.from_pretrained(
    "Shotaro30678/emotion_predictor_for_emotion_chat_bot",
    trust_remote_code=True,
)

In [53]:
emotion_predictor = pipeline(
    "sentiment-analysis",
    model=emotion_predictor_model,
    tokenizer=emotion_predictor_tokenizer,
    top_k=7,
    torch_dtype=torch.float32,
    device_map="auto",
    trust_remote_code=True,
)

In [54]:
emotion_predictor.model = torch.compile(emotion_predictor.model, mode="reduce-overhead")

In [55]:
print(emotion_predictor.model)

OptimizedModule(
  (_orig_mod): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-5): 6 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear4bit(in_features=768, out_features=768, bias=True)
                (key): Linear4bit(in_features=768, out_features=768, bias=True)
                (value): Linear4bit(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
         

### Emotion Model

In [56]:
emotion_model = EmotionModel.from_pretrained(
    "hermeschen1116/emotion_model_for_emotion_chat_bot"
)

In [57]:
emotion_model = torch.compile(emotion_model, mode="reduce-overhead")

In [58]:
print(emotion_model)

OptimizedModule(
  (_orig_mod): EmotionModel(
    (_EmotionModel__weight_Q): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__weight_K): Linear(in_features=7, out_features=7, bias=False)
    (_EmotionModel__dropout): Dropout(p=0.5, inplace=False)
    (_EmotionModel__weight_D): Linear(in_features=7, out_features=7, bias=True)
  )
)


### Similarity Analyzer

In [59]:
threshold: float = 0.5

In [60]:
similarity_analyzer = SimilarityAnalyser(threshold)

## Combine All Modules

### Initialize Text Streamer

In [61]:
streamer = TextStreamer(
    tokenizer, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [62]:
system_prompt: str = ""  # input("Enter your system prompt: ").strip()

In [63]:
chat_buffer: list = [
    {
        "role": "system",
        "content": {"emotion": "", "dialog": system_prompt},
    }
]

### Initialize Bot's Emotion Representation

In [64]:
bot_emotion_representation: Tensor = generate_dummy_representation(randint(0, 6))
bot_emotion_representation

tensor([0.7378, 0.6395, 0.9678, 0.3301, 0.2555, 0.3719, 0.9603])

In [65]:
ideal_bot_emotion_representation: Tensor = torch.tensor(
    [0], device=bot_emotion_representation.device
)
while True:
    ideal_bot_emotion_representation = generate_dummy_representation(randint(0, 6))
    if not ideal_bot_emotion_representation.eq(bot_emotion_representation).all():
        break
ideal_bot_emotion_representation

tensor([0.9431, 0.6571, 0.2676, 0.8220, 0.9937, 0.9270, 0.8024])

In [66]:
def get_top_emotion(input_text_emotion: list) -> str:
    label2score: dict = {
        emotion["label"]: emotion["score"] for emotion in input_text_emotion[0]
    }
    return max(label2score, key=label2score.get)

In [67]:
input_text: str = "Hello, how are you?"  # input("Talk to bot...").strip()
input_text_emotion: list = sentiment_analyzer(chat_buffer[-1]["content"]["dialog"])
chat_buffer.append(
    {
        "role": "user",
        "content": {
            "emotion": get_top_emotion(input_text_emotion),
            "dialog": input_text,
        },
    }
)
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}}]

In [68]:
input_text_emotion_composition: Tensor = get_sentiment_composition(input_text_emotion)
input_text_emotion_composition

tensor([0.3073, 0.1151, 0.1151, 0.1151, 0.1168, 0.1153, 0.1152])

### Update Current Bot's Emotion Representation

In [69]:
bot_emotion_representation = emotion_model.forward(
    input_text_emotion_composition, bot_emotion_representation
)
bot_emotion_representation

skipping cudagraphs due to skipping cudagraphs due to cpu device (primals_5). Found from : 
   File "/home/hermeschen/Repo/chat-bot/src/models/libs/EmotionModel.py", line 40, in forward
    decomposed_representation: Tensor = representation.diag().to(



tensor([0.4907, 0.2973, 0.2982, 0.2970, 0.2986, 0.2973, 0.2979],
       device='cuda:0', grad_fn=<CompiledFunctionBackward>)

### Generate All Possible Responses In 7 Different Emotions

In [70]:
def create_candidates_buffer(chat_buffer: list) -> list:
    emotions: list = [
        "neutral",
        "anger",
        "disgust",
        "fear",
        "happiness",
        "sadness",
        "surprise",
    ]

    candidates_buffer: list = [
        chat_buffer
        + [
            {
                "role": "bot",
                "content": {"emotion": emotion, "dialog": ""},
            }
        ]
        for emotion in emotions
    ]

    return candidates_buffer

In [71]:
candidates_buffer: list = create_candidates_buffer(chat_buffer)
candidates_buffer

[[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'neutral', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'anger', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'disgust', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
  {'role': 'bot', 'content': {'emotion': 'fear', 'dialog': ''}}],
 [{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
  {'role': 'user',
   'content': {'emotion': 

In [72]:
generated_candidates: list = [
    result[0]["generated_text"]
    for result in response_generator(candidates_buffer, streamer=streamer)
    if result[0]["generated_text"][-1]["content"]["dialog"] != ""
]
print()
pprint(generated_candidates)

   neutral   Hello, how are you?     neutral    Fine, thank you. 
   neutral   Hello, how are you?     anger    Hi! I'm fine! 
   neutral   Hello, how are you?     disgust   
   neutral   Hello, how are you?     fear    Hi! I'm scared! 
   neutral   Hello, how are you?     happiness   
   neutral   Hello, how are you?     sadness   
   neutral   Hello, how are you?     surprise     Hi! I haven't seen you for ages! 

[[{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello, how are you?', 'emotion': 'neutral'},
   'role': 'user'},
  {'content': {'dialog': ' Fine, thank you. ', 'emotion': 'neutral'},
   'role': 'bot'}],
 [{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello, how are you?', 'emotion': 'neutral'},
   'role': 'user'},
  {'content': {'dialog': " Hi! I'm fine! ", 'emotion': 'anger'},
   'role': 'bot'}],
 [{'content': {'dialog': '', 'emotion': ''}, 'role': 'system'},
  {'content': {'dialog': 'Hello

### Simulate Bot's Possible Emotion Representation

In [73]:
def get_possible_response_emotion_representation(
    candidates_buffer: list, emotion_predictor
) -> dict:
    possible_user_response_emotion: dict = {}
    for candidate in candidates_buffer:
        emotion: list = emotion_predictor(candidate[-1]["content"]["dialog"])
        possible_user_response_emotion[candidate[-1]["content"]["emotion"]] = (
            get_sentiment_composition(emotion)
        )

    return possible_user_response_emotion

In [74]:
possible_response_emotion_composition: dict = (
    get_possible_response_emotion_representation(
        generated_candidates, emotion_predictor
    )
)
possible_response_emotion_composition

{'neutral': tensor([0.1371, 0.1183, 0.1180, 0.1181, 0.2714, 0.1184, 0.1187]),
 'anger': tensor([0.1727, 0.1222, 0.1206, 0.1214, 0.2200, 0.1214, 0.1216]),
 'fear': tensor([0.2380, 0.1309, 0.1211, 0.1292, 0.1302, 0.1248, 0.1258]),
 'surprise': tensor([0.1770, 0.1285, 0.1226, 0.1249, 0.1973, 0.1238, 0.1259])}

In [75]:
simulated_emotion_representation: dict = {
    k: emotion_model.forward(v, bot_emotion_representation)
    for k, v in possible_response_emotion_composition.items()
}
simulated_emotion_representation

skipping cudagraphs due to skipping cudagraphs due to cpu device (primals_5). Found from : 
   File "/home/hermeschen/Repo/chat-bot/src/models/libs/EmotionModel.py", line 40, in forward
    decomposed_representation: Tensor = representation.diag().to(



{'neutral': tensor([0.3206, 0.3003, 0.3010, 0.2999, 0.4532, 0.3005, 0.3015],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>),
 'anger': tensor([0.3562, 0.3043, 0.3036, 0.3034, 0.4018, 0.3036, 0.3043],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>),
 'fear': tensor([0.4215, 0.3131, 0.3041, 0.3110, 0.3120, 0.3068, 0.3085],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>),
 'surprise': tensor([0.3605, 0.3105, 0.3056, 0.3068, 0.3792, 0.3059, 0.3086],
        device='cuda:0', grad_fn=<CompiledFunctionBackward>)}

### Compute Similarity Score

In [76]:
similarity_score: list = similarity_analyzer(
    list(simulated_emotion_representation.values()), ideal_bot_emotion_representation
).tolist()
similarity_score

[0.3942907750606537,
 0.3929142951965332,
 0.39094415307044983,
 0.39175572991371155]

### Get Best Response Emotion

In [77]:
emotions: list = [
    "neutral",
    "anger",
    "disgust",
    "fear",
    "happiness",
    "sadness",
    "surprise",
]
best_bot_response_emotion: str = emotions[
    max(enumerate(similarity_score), key=itemgetter(1))[0]
]
best_bot_response_emotion

'neutral'

In [78]:
chat_buffer.append(
    {"role": "bot", "content": {"emotion": best_bot_response_emotion, "dialog": ""}}
)
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
 {'role': 'bot', 'content': {'emotion': 'neutral', 'dialog': ''}}]

### Response

In [79]:
chat_buffer = response_generator(chat_buffer, streamer=streamer)[0]["generated_text"]
chat_buffer

   neutral   Hello, how are you?     neutral    Fine, thank you. 


[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
 {'role': 'bot',
  'content': {'emotion': 'neutral', 'dialog': ' Fine, thank you. '}}]

In [80]:
if len(chat_buffer) > 11:
    chat_buffer.pop(1)
chat_buffer

[{'role': 'system', 'content': {'emotion': '', 'dialog': ''}},
 {'role': 'user',
  'content': {'emotion': 'neutral', 'dialog': 'Hello, how are you?'}},
 {'role': 'bot',
  'content': {'emotion': 'neutral', 'dialog': ' Fine, thank you. '}}]